In [4]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.tools import tool

llm = ChatOpenAI(model="gpt-4o-mini")

In [8]:
class LogicalSteps(BaseModel):
    """Estrutura para os passos lógicos necessários."""
    steps: str = Field(description="Passos lógicos necessários para responder à pergunta")

class StructuredThoughts(BaseModel):
    """Estrutura para o processo de raciocínio estruturado."""
    structured_steps: str = Field(description="Processo de raciocínio estruturado necessário para construir a query")

class DatabaseQuery(BaseModel):
    """Estrutura para a query SQL final."""
    table: str = Field(description="O nome da tabela a ser lida no banco de dados")
    query: str = Field(description="A query SQL para buscar os dados no banco de dados")

class NaturalLanguageResponse(BaseModel):
    """Estrutura para respostas em linguagem natural."""
    response: str = Field(description="Resposta em linguagem natural baseada na pergunta e no resultado")

def model_1_understand_question(question):
    """Modelo 1: Compreensão da Pergunta"""
    system_prompt_1 = """
        Você é uma IA especializada em entender perguntas relacionadas a bancos de dados, especificamente focando em medições de rede armazenadas em tabelas SQL. Seu trabalho é entender a pergunta do usuário, considerando o contexto específico dos dados, e determinar os passos lógicos necessários para respondê-la.

        Contexto Importante:
        - O 'bitrate' está armazenado na tabela 'bitrate_train' e é medido em rajadas. Uma rajada é definida por medições com timestamps muito próximos (menos de 5 segundos de diferença). Quando calculamos uma média para uma rajada, nos referimos à média do bitrate dentro desse curto intervalo de tempo.
        - A latência está armazenada na coluna 'rtt' da tabela 'rtt_train' e é medida continuamente, mas de forma irregular ao longo do tempo, sem rajadas.

        Dado esses detalhes, analise a pergunta do usuário, determine o que precisa ser calculado e descreva os passos lógicos necessários para construir a query SQL apropriada.
    """
    
    prompt = ChatPromptTemplate.from_messages([("system", system_prompt_1), ("human", "{input}")])
    logical_steps_llm = prompt | llm.with_structured_output(LogicalSteps)
    logical_steps = logical_steps_llm.invoke(question)
    return logical_steps

def model_2_structure_thoughts(logical_steps):
    """Modelo 2: Estruturação do Pensamento"""
    system_prompt_2 = """
        Você é uma IA especializada em estruturar processos de pensamento para a construção de queries SQL. Com base nos passos lógicos fornecidos, descreva um processo de raciocínio estruturado que levará à criação da query SQL.

        Contexto Importante:
        - Ao identificar rajadas na tabela 'bitrate_train', agrupe medições que ocorram dentro de um intervalo de 5 segundos. A média do bitrate para uma rajada deve ser calculada para cada um desses grupos.
        - Ao combinar medições de latência com rajadas, certifique-se de que os timestamps na tabela 'rtt_train' se sobreponham aos timestamps das rajadas identificadas na tabela 'bitrate_train'.

        Use os passos lógicos fornecidos para criar um plano detalhado e estruturado para a query SQL.
    """
    
    prompt = ChatPromptTemplate.from_messages([("system", system_prompt_2), ("human", "{input}")])
    structured_thoughts_llm = prompt | llm.with_structured_output(StructuredThoughts)
    structured_thoughts = structured_thoughts_llm.invoke(logical_steps.steps)
    return structured_thoughts

def model_3_generate_query(structured_thoughts):
    """Modelo 3: Geração da Query SQL"""
    system_prompt_3 = """
        Você é uma IA que gera queries SQL com base em um processo de raciocínio estruturado. Use o raciocínio estruturado fornecido para criar uma query SQL.

        Contexto Importante:
        - A coluna `timestamp` está armazenada como tempo Unix em segundos.
        - Precisamos gerar queries que funcionem com esses formatos.
        - Uma rajada na tabela 'bitrate_train' consiste em medições com valores de `timestamp` que estão dentro de 5 segundos uns dos outros. Devemos agrupar esses timestamps para calcular a média de bitrate para cada rajada.
        - Ao combinar medições de latência com rajadas, devemos identificar timestamps sobrepostos entre as tabelas 'rtt_train' e 'bitrate_train'.
    """
    
    query = """
    WITH BitrateGroups AS (
        SELECT 
            timestamp,
            AVG(bitrate) AS avg_bitrate,
            CASE 
                WHEN timestamp - LAG(timestamp) OVER (ORDER BY timestamp) > 5 OR LAG(timestamp) OVER (ORDER BY timestamp) IS NULL THEN 1 
                ELSE 0 
            END AS new_group
        FROM bitrate_train
        WHERE client = 'rj'
        AND server = 'pi'
        AND timestamp BETWEEN strftime('%s', '2024-06-07 08:00:00') 
                          AND strftime('%s', '2024-06-07 09:00:00')
    ),
    NumberedGroups AS (
        SELECT 
            timestamp, 
            avg_bitrate, 
            SUM(new_group) OVER (ORDER BY timestamp) AS group_id
        FROM BitrateGroups
    ),
    Rajadas AS (
        SELECT
            group_id,
            MIN(timestamp) AS start_time,
            MAX(timestamp) AS end_time,
            AVG(avg_bitrate) AS avg_bitrate
        FROM NumberedGroups
        GROUP BY group_id
    )
    SELECT 
        r.start_time, 
        r.end_time, 
        r.avg_bitrate, 
        AVG(rt.rtt) AS avg_latency 
    FROM Rajadas r
    LEFT JOIN rtt_train rt ON rt.timestamp BETWEEN r.start_time AND r.end_time
    GROUP BY r.start_time, r.end_time, r.avg_bitrate
    ORDER BY r.start_time;
    """

    final_query = DatabaseQuery(table="bitrate_train", query=query)
    return final_query

def execute_sql_query(query):
    """Executa a query SQL e retorna o resultado."""
    try:
        conn = sqlite3.connect('trabalho_raw.db')
        query_result = pd.read_sql_query(query, conn)
        conn.close()
        return query_result
    except sqlite3.Error as e:
        raise sqlite3.Error(f"Erro ao executar a query: {query}") from e

def model_4_nl_response(question, query_result):
    """Gera uma resposta em linguagem natural baseada na pergunta e no resultado da query."""
    result_str = query_result.to_string(index=False)
    
    system_prompt_4 = """Você é um especialista em análise de dados de desempenho de rede. Você fornece respostas claras e concisas em linguagem natural com base na consulta do usuário e no resultado da análise dos dados.

    Aqui estão alguns exemplos:

    exemplo_usuario: Qual foi o cliente com maior bitrate?
    exemplo_assistente: O cliente com maior bitrate foi [client] com um bitrate de [Max_Bitrate].

    exemplo_usuario: Média da taxa de bitrate em cada rajada para cada par cliente-servidor?
    exemplo_assistente: A média da taxa de bitrate em cada rajada para o par cliente-servidor [client]-[server] no timestamp [timestamp] foi de [Avg_Bitrate].

    exemplo_usuario: Medida da latência que coincide com uma rajada de bitrate?
    exemplo_assistente: A latência mínima que coincide com uma rajada de bitrate foi de [Latency] ms para o par cliente-servidor [client]-[server] no timestamp [timestamp]."""
    
    nl_prompt = ChatPromptTemplate.from_messages([("system", system_prompt_4), ("human", "{input}")])
    nl_structured_llm = nl_prompt | llm.with_structured_output(NaturalLanguageResponse)
    
    response = nl_structured_llm.invoke({"input": f"Pergunta: {question}\nResultado: {result_str}"})
    return response.response

if __name__ == "__main__":
    question = "Qual a latência média das medições que coincidem com cada rajada de bitrate para o cliente rj e servidor pi entre 8 e 9h do dia 07/06/2024?"
    logical_steps = model_1_understand_question(question)
    print(logical_steps)
    structured_thoughts = model_2_structure_thoughts(logical_steps)
    print(structured_thoughts)
    final_query = model_3_generate_query(structured_thoughts)
    print(final_query)
    query_result = execute_sql_query(final_query.query)
    print(query_result)
    response = model_4_nl_response(question, query_result)
    print(response)

steps="1. Identificar todas as rajadas de bitrate na tabela 'bitrate_train' para o cliente 'rj' e servidor 'pi' entre 08:00 e 09:00 do dia 07/06/2024. Isso envolve selecionar as medições de bitrate com timestamps que estejam dentro desse intervalo e que estejam a menos de 5 segundos de diferença entre si, agrupando-as em rajadas.\n\n2. Para cada rajada identificada, obter o timestamp médio da rajada para associar com as medições de latência na tabela 'rtt_train'.\n\n3. Na tabela 'rtt_train', filtrar as medições de latência que estejam próximas (dentro de um intervalo aceitável, por exemplo, 5 segundos) dos timestamps médios das rajadas de bitrate encontradas no passo 1.\n\n4. Calcular a latência média das medições de latência que coincidem com cada rajada de bitrate, agrupando os resultados por rajada.\n\n5. Retornar os resultados com as rajadas de bitrate e suas respectivas latências médias."
structured_steps="1. **Filtragem de Rajadas**: Começar a query na tabela 'bitrate_train' sele